In [60]:
import requests as req , re , pandas as pd
from pyvis.network import Network

In [61]:
def connect(source):
    url = 'https://t.me/s/' + source
    request = req.get(url)
    html = request.text
    print('connected to' , source)
    return html

Getting links to other telegram entities 

In [62]:
def targets(html , source):
    regex = re.compile(r'href=.https://t.me/\w+')
    data = []
    for target in regex.findall(html):
        target = target.rsplit('/' , 1)[1]
        if target.lower() != source.lower():
            data.append([source, target])
    edge_df = pd.DataFrame(data , columns=['source', 'target'])
    print('got targets from' , source)
    return edge_df

In [63]:
def subs (html , source):
    regex = re.compile(r'class=.counter_value.>[^<]+')
    try:
        string = regex.findall(html)[0]
        number = convert_str_to_number(string.rsplit('>' , 1)[1])
        print('got ' , number ,  'subs from ' , source)
        return number
    except:
        print('cant get subs for' , source)
    return

Note: connect , targets and subs functions can be edited per social network

In [64]:
def convert_str_to_number(x):
    total_stars = 0
    num_map = {'K':1000, 'M':1000000, 'B':1000000000}
    if x.isdigit():
        total_stars = int(x)
    else:
        if len(x) > 1:
            total_stars = float(x[:-1]) * num_map.get(x[-1].upper(), 1)
    return int(total_stars)

In [65]:
def finals(source):
    html = connect(source)
    size = subs(html , source)
    edge_df = targets(html , source)
    edge_df['source_node_size'] = size
    edge_df['edge_size'] = edge_df.groupby(['target'])['source'].transform('count')
    edge_df = edge_df.drop_duplicates(subset=['target'])
    print('passed finals for' , source)
    return edge_df

In [66]:
def loop(df , i , start_channel_name, iter_number):
    iter = iter_number
    final_df  = df
    if i < iter:
      targets = df['target']
      for target in targets:
          edge_df = finals(target)
          final_df = final_df.append(edge_df)
          print('current length = ' , len(final_df))
      if i+1==iter:
        final_df.to_csv('{} iteration for {}.csv'.format(str(i),start_channel_name))
      i += 1
      print('iteration' , i)
      loop(final_df , i , start_channel_name, iter)
      return
    else: return "finished"

Running the data code

In [67]:
channel_name = input("Enter the channel name: ")
iterations_number = int(input("Enter the number of iterations: "))
loop(finals(channel_name) , 0 , channel_name, iterations_number)

connected to bnetanyahu
got  40400 subs from  bnetanyahu
got targets from bnetanyahu
passed finals for bnetanyahu
iteration 1


Defining the graph

In [68]:
got_net = Network(height='750px', width='100%', bgcolor='#222222', font_color='whitw' , directed=True)

Defining data for the graph

In [69]:
edge_df = pd.read_csv('{} iteration for {}.csv'.format(str(iterations_number-1), channel_name))

edge_data = zip(edge_df['source'], edge_df['target'], edge_df['edge_size'] , edge_df['source_node_size'])

Creating edges and nodes

In [70]:
for row in edge_data:
    src = row[0]
    trgt = row[1]
    w = row[2]
    size = row[3]
    
    source_node_link = "<a href=\'http://t.me/s/" + src + "'>" + src + "</a>"
    target_node_link = "<a href=\'http://t.me/s/" + trgt + "'>" + trgt + "</a>"
    
    got_net.add_node(src, src, title=source_node_link , value=size)
    got_net.add_node(trgt, trgt, title=target_node_link , value=size/10)
    got_net.add_edge(src, trgt, value=w)

Creating the graph

In [71]:
got_net.show_buttons()
got_net.show('{} iteration for {}.html'.format(str(iterations_number-1),channel_name))